In [1]:
import zipfile

zip_path = "/content/multi-class-garbage-classification.zip"
extract_path = "/content/multi-class-garbage-classification"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [ ]:
# !pip install --upgrade torch

In [18]:
import kagglehub

unified = kagglehub.dataset_download("siddhantmaji/unified-waste-classification-dataset")
unified = f"{unified}/content/unified_dataset"  # adjust based on your extraction path
print("Path to dataset files:", unified)

100%|██████████| 955M/955M [00:47<00:00, 21.0MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/siddhantmaji/unified-waste-classification-dataset/versions/1/content/unified_dataset


In [19]:
import shutil
import os

# Paths
original_dataset_dir = "/content/multi-class-garbage-classification/content/multi-class-garbage-classification"
unified_dataset_dir = unified  # already set earlier

# Classes to add
missing_classes = ["battery", "textiles"]
num_images_to_copy = 1000

for cls in missing_classes:
    src_dir = os.path.join(unified_dataset_dir, cls)
    dst_dir = os.path.join(original_dataset_dir, cls)

    # Create destination directory if it doesn't exist
    os.makedirs(dst_dir, exist_ok=True)

    # List all image files in the source class folder
    images = sorted([
        f for f in os.listdir(src_dir)
        if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))
    ])

    # Copy up to 1000 images
    for i, img_name in enumerate(images[:num_images_to_copy]):
        src_path = os.path.join(src_dir, img_name)
        dst_path = os.path.join(dst_dir, f"{cls}_{i}{os.path.splitext(img_name)[1]}")
        shutil.copyfile(src_path, dst_path)

    print(f"✅ Copied {min(len(images), num_images_to_copy)} images to {dst_dir}")

✅ Copied 1000 images to /content/multi-class-garbage-classification/content/multi-class-garbage-classification/battery
✅ Copied 1000 images to /content/multi-class-garbage-classification/content/multi-class-garbage-classification/textiles


In [20]:
import os
import torch
from torchvision import transforms, datasets

data_dir = "/content/multi-class-garbage-classification/content/multi-class-garbage-classification"
IMG_SIZE = 224
BATCH_SIZE = 32

transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load dataset using ImageFolder
dataset = datasets.ImageFolder(root=data_dir, transform=transform)

In [21]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Subset

targets = [sample[1] for sample in dataset.samples]

train_idx, val_idx = train_test_split(
    range(len(dataset)), test_size=0.2, stratify=targets, random_state=42
)

train_dataset = Subset(dataset, train_idx)
val_dataset = Subset(dataset, val_idx)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

In [27]:
import torch.nn as nn
from torchvision import models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = torch.load("/content/mobilenetv3_garbage_classifier.pt", weights_only=False)
model = model.to(device)

In [28]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [29]:
from tqdm import tqdm
import copy

EPOCHS = 30
patience = 3
best_val_loss = float("inf")
epochs_without_improvement = 0
best_model_weights = copy.deepcopy(model.state_dict())

for epoch in range(EPOCHS):
    # --- Training ---
    model.train()
    train_loss, train_correct, train_total = 0.0, 0, 0

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [Train]"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, preds = outputs.max(1)
        train_correct += (preds == labels).sum().item()
        train_total += labels.size(0)

    train_acc = 100 * train_correct / train_total
    avg_train_loss = train_loss / len(train_loader)

    # --- Validation ---
    model.eval()
    val_loss, val_correct, val_total = 0.0, 0, 0

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [Val]"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, preds = outputs.max(1)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)

    val_acc = 100 * val_correct / val_total
    avg_val_loss = val_loss / len(val_loader)

    # --- Print Epoch Summary ---
    print(
        f"Epoch {epoch+1}: "
        f"Train Loss = {avg_train_loss:.4f} | Train Acc = {train_acc:.2f}% || "
        f"Val Loss = {avg_val_loss:.4f} | Val Acc = {val_acc:.2f}%"
    )

    # --- Early Stopping Check ---
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        best_model_weights = copy.deepcopy(model.state_dict())
        epochs_without_improvement = 0
        print("✅ Validation loss improved — saving best weights.")
    else:
        epochs_without_improvement += 1
        print(f"⚠️ No improvement for {epochs_without_improvement} epoch(s).")

        if epochs_without_improvement >= patience:
            print(f"⏹️ Early stopping at epoch {epoch+1}. Restoring best model.")
            model.load_state_dict(best_model_weights)
            break

Epoch 1/30 [Val]: 100%|██████████| 50/50 [00:04<00:00, 10.68it/s]


Epoch 1: Train Loss = 0.2189 | Train Acc = 94.38% || Val Loss = 0.0754 | Val Acc = 97.81%
✅ Validation loss improved — saving best weights.


Epoch 2/30 [Val]: 100%|██████████| 50/50 [00:04<00:00, 10.47it/s]


Epoch 2: Train Loss = 0.0347 | Train Acc = 98.83% || Val Loss = 0.0712 | Val Acc = 98.00%
✅ Validation loss improved — saving best weights.


Epoch 3/30 [Val]: 100%|██████████| 50/50 [00:04<00:00, 10.39it/s]


...

Epoch 4: Train Loss = 0.0135 | Train Acc = 99.62% || Val Loss = 0.0640 | Val Acc = 98.38%
⚠️ No improvement for 1 epoch(s).


Epoch 5/30 [Val]: 100%|██████████| 50/50 [00:04<00:00, 11.54it/s]


Epoch 5: Train Loss = 0.0088 | Train Acc = 99.72% || Val Loss = 0.0601 | Val Acc = 98.62%
⚠️ No improvement for 2 epoch(s).


Epoch 6/30 [Val]: 100%|██████████| 50/50 [00:04<00:00, 11.68it/s]

Epoch 6: Train Loss = 0.0072 | Train Acc = 99.78% || Val Loss = 0.0788 | Val Acc = 98.38%
⚠️ No improvement for 3 epoch(s).
⏹️ Early stopping at epoch 6. Restoring best model.


In [30]:
from sklearn.metrics import classification_report

all_preds = []
all_labels = []

model.eval()
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Generate report
target_names = dataset.classes  # from ImageFolder — maps class indices to names
report = classification_report(all_labels, all_preds, target_names=target_names, digits=3)
print(report)

                 precision    recall  f1-score   support

        battery      1.000     1.000     1.000       200
          glass      0.970     0.965     0.967       200
          metal      0.970     0.985     0.978       200
  organic_waste      0.990     0.985     0.987       200
paper_cardboard      0.995     1.000     0.998       200
        plastic      0.971     0.990     0.980       200
       textiles      1.000     0.990     0.995       200
          trash      0.990     0.970     0.980       200

       accuracy                          0.986      1600
      macro avg      0.986     0.986     0.986      1600
   weighted avg      0.986     0.986     0.986      1600



In [31]:
torch.save(model, "/content/mobilenetv3_garbage_classifier_2.pt")